# GitFlow - Introduction

This notebook assumes you've gone through the previous notebooks, especially

- [Create Projects](11_create_projects.ipynb)

Here, we dive into [semver](https://semver.org) applied with [GitFlow](https://www.atlassian.com/git/tutorials/comparing-workflows/gitflow-workflow). A basic understanding of these two concepts is a precondition to be able to crasp the essence of this notebook.

We setup a `lib` and an `app` project, both following GitFlow. The application is, again, using the library (aka 'requiring' `lib`). In this notebook, we implement a feature on `lib` and see how this affects our `app`...

## Preparation
As shown in preceeding notebooks, we first have to build the infrastructure. This notebook assumes the reader has worked through the other notebook and therefore it's not being explained here again.

In [ ]:
from cicd_sim import *
bitbucket = Repos()
artifactory = Artifactory()
jenkins = Jenkins(artifactory, bitbucket)
# ensure 'random' generates the same number each time this notebook is run
import random
random.seed(123)

## Setup Repos
Also with no explanation, we are now going to create the repos for `lib` and `app`.

One noteworthy difference to the previous notebooks is that we're now starting with the `develop` branch rather than `master`. This makes sense because in this notebook, we're following Gitflow.

Each branch can be given a color to enhance readability. The library used to colorize is documented here: https://pypi.org/project/colored/

In [ ]:
lib = bitbucket.create_repo('lib')
lib_dev = lib.checkout('develop', 'magenta')

app = bitbucket.create_repo('app')
app_dev = app.checkout('develop', 'cyan')

## Application Version
We choose the version of `app` rather arbitrary, as it doesn't matther that much for the sake of this notebook

In [ ]:
app_dev.set_version('0.0.1')

## Library Version
We set the version of `lib` to `1.0.0`. In the context of GitFlow, the work going into the `develop` branch will become "the next release". Therefore, chosing `1.0.0` means that developers are now working "towards" version `1.0.0`. (Note: this rule is *not* set into stone and is just one possible way how to put semver and GitFlow into practice)

In [ ]:
lib_dev.set_version('1.0.0')

## Setup Application to use particular version of `lib`
The application developers need to decide which version of `lib` to use. They may decide to use a very specfic version or a version range. In the latter case, Conan will choose "the best" version according to semver rules.

Here are the most important options the `app` developer have:

- Use an explicit version, e.g. `1.2.3+20020315161502`
- Fix `major` and `minor` but automatically pick up the latest bugfix version, e.g. `1.2.x`
- Fix only `major`, means: `app` will automatically pick up the latest features and bufixes for a partical API version, e.g. `1.x`

In this example, the developers are "brave developers" which means that they always pick up the "latest and greatest" version with a compatible API version available, i.e. `1.x`

The following line tells Conan to use "the highest version with major API version 1"

In [ ]:
# due to bugs in Conan's semver handling, the following doesn't properly consider "pre-releases"
#    app_dev.set_requires('lib/1.x')
# The following is slightly less readable, but works as expected:
app_dev.set_requires('lib/>1.0.0-0') # is roughly the same as 'lib/1.x'

## Developing a feature in `lib`
As we've setup the two projects, `lib` and `app`, we're now ready to start actual development.

One of the `lib` developers decides to implement feature "A"

In [ ]:
feature_a = lib_dev.checkout('feature/a', 'yellow') # similar to `git checkout -b feature/a`
feature_a.commit_file('file_a', 'The file content') # similar to `git add file_a && git commit -m "..."`
feature_a.push()

Pushing the new feature automatically triggers a build at Jenkins. That build serves two purposes:

- It tells the developer whether his features "builds" and "tests" with no errors
- It tells BitBucket whether this feature branch is error free and therefore is allowed to be merged back to `develop` branch

## Merge feature to `develop`
In this case, the feature branch is error free and can be merged back to `develop`. The developer does so and finally pushes the `develop` branch to BitBucket... here, things start getting interesting:

In [ ]:
lib_dev.merge(feature_a)
lib_dev.push()

## Our first `lib` artifact
What happenned? First of all, pushing a branch "builds it" as was already the case for the feature branch before, indicated by

```
BUILDING            lib[develop]
```

This time, as this was a push on the `develop` branch, the buildmachine published an artifact to Artfactory. This was indicated by the following line:

```
        PUBLISH     lib[develop]               + lib/1.0.0-200206113900+0000006
```

The '+' sign indicates a new artifact being put to Artifactory.

## (Re-)building `app`
But there's more. Because a new artifact was pushed to Artifactory, the `app`'s `develop` branch was built, because that branch was setup to consume `lib` artifacts in the version range of `1.x`. The build is indicated by the following two line:

```
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200206113900+0000006
```

Very important to notice here, is that Conan did install (denoted by a '*') the previously built artifact of `lib`: `lib/1.0.0-200206113900+0000006`.

Finally, building `app` also produces an artifact which get's published at Artifactory, as indicated by the following line:

```
        PUBLISH     app[develop]               + app/0.0.1-200209125600+0000004
```

Therefore, we'd expect a total of two artifact being at Artifactory, which can be checked by the following command:

In [ ]:
artifactory.get_artifacts()